# Phase 0 — Stage 3: DoE Parameter Optimization

Uses **L16 fractional factorial design** to find optimal values for:
- **κ** (kappa): decay acceleration coefficient [0.5, 2.5]
- **θ** (theta): anomaly threshold [0.35, 0.60]
- **β** (beta): softmin sharpness [0.5, 2.0]
- **Bloom interval**: days between Bloom events [7, 14]

16 experiments instead of the full 576-combination grid search.

**Stage 3 improvements applied:**
- E(v) now includes partner-diversity entropy (detects loop attacks)
- T(v) is bell-curve shaped (penalizes hyper-active nests too)
- D(v) network-distance score available for T010

In [ ]:
import sys
sys.path.insert(0, '/Users/kunimitsu/Projects/Meguri_pre3')

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('dark_background')
RESULTS_DIR = Path('/Users/kunimitsu/Projects/Meguri_pre3/results')
DOE_DIR = RESULTS_DIR / 'doe'
print('Ready.')

## 1. Run DoE L16 (or load existing results)

In [ ]:
doe_path = DOE_DIR / 'doe_l16_results.json'

if doe_path.exists():
    with open(doe_path) as f:
        doe_results = json.load(f)
    print(f'Loaded {len(doe_results)} DoE runs from {doe_path}')
else:
    print('Running DoE L16 (this takes ~5-10 minutes)...')
    from simulation.src.doe_optimizer import run_doe_l16
    doe_results = run_doe_l16()
    print('DoE complete.')

## 2. Results Table

In [ ]:
rows = []
for r in doe_results:
    p = r['params']
    # Aggregate metrics across test cases
    metrics = r['metrics']
    avg_roi = np.mean([m.get('sybil_roi', 0) for m in metrics if m.get('attacker_count', 0) > 0] or [0])
    avg_fpr = np.mean([m.get('fpr', 0) for m in metrics])
    avg_tpr = np.mean([m.get('tpr', 0) for m in metrics if m.get('attacker_count', 0) > 0] or [0])
    avg_gini = np.mean([m.get('gini', 0) for m in metrics])
    rows.append({
        'Run': r['run'],
        'κ': p['kappa'],
        'θ': p['theta'],
        'β': p['beta'],
        'Bloom': p['bloom_interval'],
        'Score': r['score'],
        'ROI': avg_roi,
        'FPR': avg_fpr,
        'TPR': avg_tpr,
        'Gini': avg_gini,
    })

df = pd.DataFrame(rows).sort_values('Score')

def highlight_best(s):
    return ['background-color: #1a3a1a' if v == s.min() else '' for v in s]

print('DoE Results (sorted by composite score — lower is better):')
df.style.format({
    'κ': '{:.2f}', 'θ': '{:.2f}', 'β': '{:.2f}',
    'Score': '{:.4f}', 'ROI': '{:.4f}',
    'FPR': '{:.2%}', 'TPR': '{:.2%}', 'Gini': '{:.4f}'
}).apply(highlight_best, subset=['Score'])

## 3. Main Effects Plot

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('Main Effects: Each Parameter vs Metrics', fontsize=14, fontweight='bold')

param_cols = ['κ', 'θ', 'β', 'Bloom']
metric_cols = ['Score', 'ROI', 'FPR', 'TPR']
colors = ['#ff6b6b', '#ffd93d', '#6bcb77', '#4d96ff']

for col_idx, param in enumerate(param_cols):
    for row_idx, metric in enumerate(['Score', 'FPR']):
        ax = axes[row_idx, col_idx]
        unique_vals = sorted(df[param].unique())
        means = [df[df[param] == v][metric].mean() for v in unique_vals]
        ax.bar([str(round(v, 2)) for v in unique_vals], means, color=colors[col_idx], alpha=0.8)
        ax.set_title(f'{param} → {metric}', fontsize=9)
        ax.grid(alpha=0.3)
        if metric == 'FPR':
            ax.axhline(y=0.01, color='red', linestyle='--', alpha=0.5, label='1% target')
            ax.legend(fontsize=7)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'doe_main_effects.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved doe_main_effects.png')

## 4. Best Parameters

In [ ]:
best = doe_results[0]  # already sorted
best_params = best['params']

print('=== Best Parameters from DoE L16 ===')
for k, v in best_params.items():
    print(f'  {k:20s}: {v}')
print(f'\n  Composite score : {best["score"]:.4f}')

# Metrics detail
print('\n  Detailed metrics:')
for m in best['metrics']:
    print(f"  [{m.get('test_id','')}] ROI={m.get('sybil_roi',0):.4f}  "
          f"TPR={m.get('tpr',0):.2%}  FPR={m.get('fpr',0):.2%}  Gini={m.get('gini',0):.4f}")

## 5. Sensitivity Analysis (±10% perturbation)

In [ ]:
from simulation.src.runner import SimulationRunner
from simulation.src.config import TestCase

runner = SimulationRunner()
tc_base = TestCase('SENS', 1000, 10, 'A', 'Sensitivity test', '')

base_kappa = best_params['kappa']
base_theta = best_params['theta']
base_beta  = best_params['beta']
base_bloom = int(best_params['bloom_interval'])

def run_perturbed(param_name, delta_pct, **override):
    m = runner.run_single_test_case(
        TestCase(f'SENS_{param_name}_{delta_pct:+d}',
                 1000, 10, 'A', f'Sensitivity {param_name}{delta_pct:+d}%', ''),
        kappa=base_kappa, theta=base_theta, beta=base_beta, bloom_interval=base_bloom,
        verbose=False, **override
    )
    return m

print('Running sensitivity analysis (±10% on each parameter)...')
sens_rows = []

for param, base_val in [('kappa', base_kappa), ('theta', base_theta), ('beta', base_beta)]:
    for delta in [-10, +10]:
        perturbed = base_val * (1 + delta / 100)
        m = run_perturbed(param, delta, **{param: perturbed})
        sens_rows.append({
            'Parameter': param,
            'Change': f'{delta:+d}%',
            'New value': round(perturbed, 3),
            'ROI': m.get('sybil_roi', 0),
            'FPR': m.get('fpr', 0),
            'TPR': m.get('tpr', 0),
        })

df_sens = pd.DataFrame(sens_rows)
print('\nSensitivity results:')
df_sens.style.format({'ROI': '{:.4f}', 'FPR': '{:.2%}', 'TPR': '{:.2%}', 'New value': '{:.3f}'})

## 6. T010: Distance Axis Comparison

In [ ]:
t010_path = DOE_DIR / 't010_distance_axis.json'

if t010_path.exists():
    with open(t010_path) as f:
        t010 = json.load(f)
else:
    print('Running T010...')
    from simulation.src.scenario_t010 import run_t010
    t010 = run_t010(
        kappa=best_params['kappa'],
        theta=best_params['theta'],
        beta=best_params['beta'],
    )

fig, ax = plt.subplots(figsize=(10, 5))
ks = [r['k'] for r in t010['three_layer']]
roi_3 = [r['sybil_roi'] for r in t010['three_layer']]
roi_4 = [r['sybil_roi'] for r in t010['four_layer']]
thresholds = [1/k for k in ks]

ax.plot(ks, roi_3, 'o-', label='3-layer (no distance)', color='#ff6b6b', markersize=8)
ax.plot(ks, roi_4, 's-', label='4-layer (+ distance)', color='#4d96ff', markersize=8)
ax.plot(ks, thresholds, 'k--', label='Target: 1/k', alpha=0.6)
ax.set_xlabel('Attacker nests (k)')
ax.set_ylabel('Sybil ROI')
ax.set_title('T010: 3-layer vs 4-layer — Distance Axis Value')
ax.legend()
ax.grid(alpha=0.3)
ax.set_yscale('log')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 't010_distance_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n3-layer vs 4-layer ROI comparison:')
print(f'{"k":>5}  {"3-layer":>10}  {"4-layer":>10}  {"Reduction":>10}')
for k, r3, r4 in zip(ks, roi_3, roi_4):
    red = (r3 - r4) / max(r3, 1e-9) * 100
    print(f'{k:>5}  {r3:>10.4f}  {r4:>10.4f}  {red:>9.1f}%')

## 7. Final Parameter Recommendations

In [ ]:
print('='*60)
print('PHASE 0 — RECOMMENDED PARAMETERS')
print('='*60)
print()
print(f'  κ  (kappa, decay acceleration) : {best_params["kappa"]}')
print(f'  θ  (theta, anomaly threshold)  : {best_params["theta"]}')
print(f'  β  (beta, softmin sharpness)   : {best_params["beta"]}')
print(f'  Bloom interval                 : {best_params["bloom_interval"]} days')
print(f'  Deterministic ratio            : 75% / 25%')
print(f'  Grace Period                   : 30 days (confirmed)')
print(f'  Community analysis cycle       : Monthly (30 days)')
print(f'  Community algorithm            : Louvain')
print()
print('Distance axis (T010):')
if t010_path.exists():
    roi3_k10 = next(r['sybil_roi'] for r in t010['three_layer'] if r['k'] == 10)
    roi4_k10 = next(r['sybil_roi'] for r in t010['four_layer'] if r['k'] == 10)
    reduction = (roi3_k10 - roi4_k10) / max(roi3_k10, 1e-9) * 100
    if reduction > 20:
        print(f'  RECOMMENDED: Add distance axis (+{reduction:.0f}% ROI reduction at k=10)')
    else:
        print(f'  OPTIONAL: Distance axis reduces ROI by {reduction:.0f}% at k=10 (marginal benefit)')
else:
    print('  T010 not yet run.')
print()
print('='*60)
print('Phase 0 checklist status: See SIMULATION_STATUS.md')
print('='*60)